In [2]:
pip install simpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import random
import simpy

class Organism:
    def __init__(self, env, size):
        self.env = env
        self.size = size
        self.hunger = 0
        self.dead = False
        
    def eat(self, food=None):
        if food is not None:
            self.hunger -= food.size
            if self.hunger < 0:
                self.hunger = 0
    
    def die(self):
        self.dead = True
        
class Plant(Organism):
    def __init__(self, env):
        super().__init__(env, 1)
        self.process = env.process(self.grow())
        
    def grow(self):
        while not self.dead:
            yield self.env.timeout(1)
            self.size += 1
    
class Herbivore(Organism):
    def __init__(self, env, size):
        super().__init__(env, size)
    
    def eat(self, food=None):
        if isinstance(food, Plant):
            self.hunger = 0
            return
        super().eat(food)
    
class Predator(Organism):
    def __init__(self, env, size):
        super().__init__(env, size)
    
    def eat(self, food=None):
        if isinstance(food, Herbivore) or isinstance(food, Predator):
            self.hunger = 0
            return
        super().eat(food)

class Ecosystem:
    def __init__(self, env, num_plants, num_herbivores, num_predators):
        self.env = env
        self.plants = [Plant(env) for i in range(num_plants)]
        self.herbivores = [Herbivore(env, 5) for i in range(num_herbivores)]
        self.predators = [Predator(env, 10) for i in range(num_predators)]
        self.interactions = {
            'plants': {'herbivores': self.herbivores},
            'herbivores': {'plants': self.plants, 'predators': self.predators},
            'predators': {'herbivores': self.herbivores}
        }
    
    def run(self, duration):
        for i in range(duration):
            for organism_type, interactions in self.interactions.items():
                for organism in getattr(self, organism_type):
                    organism.hunger += 1
                    for other_type, other_organisms in interactions.items():
                        if other_type != organism_type:
                            prey = random.choice(other_organisms)
                            if organism_type == 'herbivores':
                                if isinstance(prey, Plant) and prey.size > 0:
                                    organism.eat(prey)
                                elif isinstance(prey, Herbivore) and prey.size > 0 and prey.size > organism.size:
                                    organism.eat(prey.plants[0])
                            elif organism_type == 'predators' and prey.size > 0 and prey.size < organism.size:
                                organism.eat(prey)

# Create the simulation environment
env = simpy.Environment()

# Create the ecosystem with 10 plants, 5 herbivores, and 2 predators
ecosystem = Ecosystem(env, 10, 5, 2)

# Run the simulation for 10 time units
ecosystem.run(10)

In [4]:
# Print the status of each organism
print("Organism status:")
for organism in ecosystem.plants + ecosystem.herbivores + ecosystem.predators:
    print(f"{organism.__class__.__name__}: {organism.size} size, {organism.hunger} hunger")

Organism status:
Plant: 1 size, 10 hunger
Plant: 1 size, 10 hunger
Plant: 1 size, 10 hunger
Plant: 1 size, 10 hunger
Plant: 1 size, 10 hunger
Plant: 1 size, 10 hunger
Plant: 1 size, 10 hunger
Plant: 1 size, 10 hunger
Plant: 1 size, 10 hunger
Plant: 1 size, 10 hunger
Herbivore: 5 size, 0 hunger
Herbivore: 5 size, 0 hunger
Herbivore: 5 size, 0 hunger
Herbivore: 5 size, 0 hunger
Herbivore: 5 size, 0 hunger
Predator: 10 size, 0 hunger
Predator: 10 size, 0 hunger
